In [1]:
#Compute a Photon Transfer Curve using a script that takes pairs of images at increasing exposure times. 

In [2]:
import sys
import os
import asyncio
import logging
import time

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [3]:
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/sashabrownsberger/ospl.xml"

In [4]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [5]:
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)

Read historical data in 0.01 sec
Read historical data in 0.26 sec
Read historical data in 0.44 sec
Read historical data in 0.68 sec
Read historical data in 1.99 sec
Read historical data in 2.47 sec
Read historical data in 3.35 sec
Read historical data in 3.88 sec
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 77 messages
Read historical data in 4.12 sec
Read historical data in 4.68 sec
Read historical data in 6.09 sec
RemoteTelemetry(ATDome, 0, position) falling behind; read 30 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 19 messages
RemoteEvent(ATDome, 0, doorEncoderExtremes) falling behind; read 34 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 22 messages
RemoteEvent(ATMCS, 0, target) fal

In [6]:
await asyncio.gather(attcs.start_task, latiss.start_task)
await attcs.enable()
await latiss.enable() 

Gathering settings.
No settings for atmcs.
Using                                                                                                                                 from settingVersions event.
No settings for atptg.
Couldn't get settingVersions event. Using empty settings.
No settings for ataos.
Using current from settingVersions event.
No settings for atpneumatics.
Using                                                                                                                                 from settingVersions event.
No settings for athexapod.
Using current from settingVersions event.
No settings for atdome.
Using test from settingVersions event.
No settings for atdometrajectory.
Using  from settingVersions event.
Settings versions: {'atmcs': '                                                                                                                               ', 'atptg': '', 'ataos': 'current', 'atpneumatics': '                                              

In [7]:
# !!!!!!! STOP !!!!!! 
#If the system is not already in position for flat fields and you would like to use flats as the basis for your PTC,
# execute this cell.  Otherwise, don't bother.
# !!!!! IF DOME IS ALREADY IN FLAT FIELD POSITION, YOU DONT'T NEED TO RUN THIS CELL 
await attcs.prepare_for_flatfield()

Cover state <MirrorCoverState.CLOSED: 6>
Opening M1 cover.
Cover state <MirrorCoverState.CLOSED: 6>
Cover state <MirrorCoverState.INMOTION: 8>
Cover state <MirrorCoverState.OPENED: 7>
Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = -045.935 | delta Az= -154.361
[Telescope] delta Alt = -044.982 | delta Az= -157.132
[Telescope] delta Alt = -043.983 | delta Az= -161.127
[Telescope] delta Alt = -041.632 | delta Az= -165.128
[Telescope] delta Alt = -037.634 | delta Az= -169.127
[Telescope] delta Alt = -033.636 | delta Az= -173.128
[Telescope] delta Alt = -029.635 | delta Az= -177.129
[Telescope] delta Alt = -025.651 | delta Az= +178.873
[Telescope] delta Al

In [8]:
#Select your filter and grating
filter_str = 'empty_1'
grating_str = 'empty_4'

In [9]:
#Define your exposure times. 
#If you are not sure about your exposure times, consider taking a set with only one exposure time and basing your scaling off of that 
# (see the DisplayImage.ipynb for instructions on pulling up the flat field image that you produce). 
exp_times = [0.25 * scaling for scaling in [1.0] + np.arange(2.0, 20.5, 2.0).tolist()]
print ('exposure times are ' + str(exp_times))

exposure times are [0.25, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


In [10]:
for exp_time in exp_times: 
    PTC_pairs = await latiss.take_flats(exptime=exp_time, nflats=2, filter=filter_str, grating=grating_str)
    print ('The PTC pair data ids for exposure time ' +str(exp_time) + ' are: '+  str(PTC_pairs)) 

Generating group_id
FLAT 0001 - 0002
Waiting for header service LFO before continuing
FLAT 0002 - 0002
Waiting for header service LFO before continuing
The PTC pair data ids for exposure time 0.25 are: [2020031200011 2020031200012]
Generating group_id
FLAT 0001 - 0002
Waiting for header service LFO before continuing
FLAT 0002 - 0002
Waiting for header service LFO before continuing
The PTC pair data ids for exposure time 0.5 are: [2020031200013 2020031200014]
Generating group_id
FLAT 0001 - 0002
Waiting for header service LFO before continuing
FLAT 0002 - 0002
Waiting for header service LFO before continuing
The PTC pair data ids for exposure time 1.0 are: [2020031200015 2020031200016]
Generating group_id
FLAT 0001 - 0002
Waiting for header service LFO before continuing
FLAT 0002 - 0002
Waiting for header service LFO before continuing
The PTC pair data ids for exposure time 1.5 are: [2020031200017 2020031200018]
Generating group_id
FLAT 0001 - 0002
Waiting for header service LFO before 

In [11]:
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend("firefly")
os.environ['FIREFLY_URL'] = "http://firefly01.cp.lsst.org:8080/firefly/"

In [12]:
from lsst.ip.isr.isrTask import IsrTask

isrConfig = IsrTask.ConfigClass()
isrConfig.doLinearize = False
isrConfig.doBias = False
isrConfig.doFlat = False
isrConfig.doDark = False
isrConfig.doFringe = False
isrConfig.doDefect = False
isrConfig.doAddDistortionModel = False
isrConfig.doWrite = False

isrTask = IsrTask(config=isrConfig)

/opt/lsst/software/stack/python/miniconda3-4.7.12/envs/lsst-scipipe-984c9f7/lib/python3.7/site-packages/ipykernel/__main__.py:10: FutureWarning: Config field doAddDistortionModel is deprecated: Camera geometry is incorporated when reading the raw files. This option no longer is used, and will be removed after v19.


In [13]:
butler = dafPersist.Butler("/project/shared/auxTel")
dataId = dict(dayObs="2020-03-12") #Your observing date 

In [14]:
dataId["seqNum"] = 11
exp = butler.get('raw', dataId)

Using translation class LSST_LATISS
Checking for header correction file named LATISS-AT_O_20200312_000011.yaml
AT_O_20200312_000011: Forcing detector serial to ITL-3800C-068
AT_O_20200312_000011: Forcing SHUTTIME header to be None
AT_O_20200312_000011: Forcing blank RADESYS to 'ICRS'
Using translation class LSST_LATISS
Checking for header correction file named LATISS-AT_O_20200312_000011.yaml
AT_O_20200312_000011: Forcing detector serial to ITL-3800C-068
Using translation class LSST_LATISS
AT_O_20200312_000011: Dark time less than exposure time. Setting dark time to the exposure time.


In [15]:
disp = afwDisplay.Display(1, name='sample', reopenPlot=True)
disp.scale('asinh', 'zscale')
disp.mtv(exp, title=f"{dataId}")

Starting new HTTPS connection (1): summit-lsp.lsst.codes:443
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.co